In [ ]:
'''
Single Server
'''


import cv2
import time
import base64
import asyncio
import websockets
import numpy as np
from threading import Thread
from djitellopy import Tello
from pynq_dpu import DpuOverlay


class TelloDpuServer:
    def __init__(self, dpu_bitfile, model_file, video_port=5678, command_port=5679, dis=25, ang=30):
        self.dis_scale = dis
        self.ang_scale = ang
        # 硬件初始化
        self.overlay = DpuOverlay(dpu_bitfile)
        self.overlay.load_model(model_file)
        self.dpu = self.overlay.runner

        # 获取输入输出tensor
        self.inputTensors = self.dpu.get_input_tensors()
        self.outputTensors = self.dpu.get_output_tensors()

        # 定义输入输出形状
        self.shapeIn = tuple(self.inputTensors[0].dims)
        self.shapeOut = tuple(self.outputTensors[0].dims)

        # 创建输入输出数据
        self.input_data = [np.empty(self.shapeIn, dtype=np.float32, order="C")]
        self.output_data = [np.empty(self.shapeOut, dtype=np.float32, order="C")]

        # 连接Tello无人机
        self.tello = Tello(scale=6)
        self.tello.connect()
        self.tello.streamon()

        # WebSocket配置
        self.video_port = video_port
        self.command_port = command_port

    async def handle_commands(self, websocket, path):
        """
        处理来自客户端的控制命令。
        """
        while True:
            try:
                message = await asyncio.wait_for(websocket.recv(), timeout=0.1)
                if message == "takeoff":
                    self.tello.takeoff()
                elif message == "land":
                    self.tello.land()
                elif message == "moveLeft":
                    self.tello.move_left(self.dis_scale)
                elif message == "moveRight":
                    self.tello.move_right(self.dis_scale)
                elif message == "moveForward":
                    self.tello.move_forward(self.dis_scale)
                elif message == "moveBackward":
                    self.tello.move_back(self.dis_scale)
                elif message == "ascend":
                    self.tello.move_up(self.dis_scale)
                elif message == "descend":
                    self.tello.move_down(self.dis_scale)
                elif message == "rotateClockwise":
                    self.tello.rotate_clockwise(self.ang_scale)
                elif message == "rotateCounterclockwise":
                    self.tello.rotate_counter_clockwise(self.ang.dis_scale)
            except asyncio.TimeoutError:
                # 没有接收到消息，跳过并继续等待
                pass

    async def video_stream(self, websocket, path):
        """
        循环获取视频帧并处理。
        """
        while True:
            # 获取Tello视频帧并处理
            frame1 = cv2.resize(self.tello.get_frame_read().frame, (256, 256))
            in_img = cv2.cvtColor(np.array(frame1, dtype='float32'), cv2.COLOR_RGB2BGR)
            self.input_data[0] = in_img

            # 模型预测
            job_id = self.dpu.execute_async(self.input_data, self.output_data)
            self.dpu.wait(job_id)

            # 获取模型预测后的帧
            frame2 = np.array(self.output_data).squeeze()

            # 转换颜色空间和裁剪
            rgb_frame2 = frame2.clip(0, 255)

            # 编码视频帧
            _, end2 = cv2.imencode('.jpg', rgb_frame2)

            # 进行base64编码
            jpg2_as_text = base64.b64encode(end2).decode('utf-8')

            # 将视频帧发送给客户端
            await websocket.send(jpg2_as_text)

    def run_video_server(self):
        """
        启动视频流 WebSocket 服务器的独立线程。
        """
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        server = websockets.serve(self.video_stream, "localhost", self.video_port)
        loop.run_until_complete(server)
        print(f"视频流服务器已启动，端口：{self.video_port}")
        loop.run_forever()

    def run_command_server(self):
        """
        启动控制指令 WebSocket 服务器的独立线程。
        """
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        server = websockets.serve(self.handle_commands, "localhost", self.command_port)
        loop.run_until_complete(server)
        print(f"控制指令服务器已启动，端口：{self.command_port}")
        loop.run_forever()

    def start(self):
        """
        使用多线程分别启动视频流和控制指令服务器。
        """
        video_thread = Thread(target=self.run_video_server)
        command_thread = Thread(target=self.run_command_server)
        video_thread.start()
        command_thread.start()
        video_thread.join()
        command_thread.join()


# 仅在Jupyter环境中运行
server = TelloDpuServer(dpu_bitfile="dpu.bit", model_file="sesr_model_best.xmodel")
server.start()

[INFO] tello.py - 131 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 439 - Send command: 'command'
[ERROR] tello.py - 459 - 'utf-8' codec can't decode byte 0xcc in position 0: invalid continuation byte
[INFO] tello.py - 439 - Send command: 'command'
[INFO] tello.py - 463 - Response command: 'ok'
[INFO] tello.py - 439 - Send command: 'streamon'
[INFO] tello.py - 463 - Response streamon: 'ok'


控制指令服务器已启动，端口：5679
视频流服务器已启动，端口：5678


[INFO] tello.py - 439 - Send command: 'takeoff'
[INFO] tello.py - 463 - Response takeoff: 'ok'
[INFO] tello.py - 439 - Send command: 'cw 30'
[INFO] tello.py - 463 - Response cw 30: 'ok'
connection handler failed
Traceback (most recent call last):
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/websockets/legacy/server.py", line 232, in handler
    await self.ws_handler(self)
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/websockets/legacy/server.py", line 1146, in _ws_handler
    return await cast(
  File "/tmp/ipykernel_17730/3231302380.py", line 73, in handle_commands
    self.tello.rotate_counter_clockwise(ang.dis_scale)
NameError: name 'ang' is not defined


In [ ]:
'''
Double Server
'''


import cv2
import base64
import asyncio
import websockets
import numpy as np
from threading import Thread
from djitellopy import Tello
from pynq_dpu import DpuOverlay


class TelloDpuServer:
    def __init__(self, dpu_bitfile, model_file, video_port=5678, command_port=5679, dis=25, ang=30):
        self.dis_scale = dis
        self.ang_scale = ang
        # 硬件初始化
        self.overlay = DpuOverlay(dpu_bitfile)
        self.overlay.load_model(model_file)
        self.dpu = self.overlay.runner

        # 获取输入输出tensor
        self.inputTensors = self.dpu.get_input_tensors()
        self.outputTensors = self.dpu.get_output_tensors()

        # 定义输入输出形状
        self.shapeIn = tuple(self.inputTensors[0].dims)
        self.shapeOut = tuple(self.outputTensors[0].dims)

        # 创建输入输出数据
        self.input_data = [np.empty(self.shapeIn, dtype=np.float32, order="C")]
        self.output_data = [np.empty(self.shapeOut, dtype=np.float32, order="C")]

        # 连接Tello无人机
        self.tello = Tello(scale=8)
        self.tello.connect()
        self.tello.streamon()

        # WebSocket配置
        self.video_port = video_port
        self.command_port = command_port

    async def handle_commands(self, websocket, path):
        """
        处理来自客户端的控制命令。
        """
        while True:
            try:
                message = await asyncio.wait_for(websocket.recv(), timeout=0.1)
                if message == "takeoff":
                    self.tello.takeoff()
                elif message == "land":
                    self.tello.land()
                elif message == "moveLeft":
                    self.tello.move_left(self.dis_scale)
                elif message == "moveRight":
                    self.tello.move_right(self.dis_scale)
                elif message == "moveForward":
                    self.tello.move_forward(self.dis_scale)
                elif message == "moveBackward":
                    self.tello.move_back(self.dis_scale)
                elif message == "ascend":
                    self.tello.move_up(self.dis_scale)
                elif message == "descend":
                    self.tello.move_down(self.dis_scale)
                elif message == "rotateClockwise":
                    self.tello.rotate_clockwise(self.ang_scale)
                elif message == "rotateCounterclockwise":
                    self.tello.rotate_counter_clockwise(self. ang.dis_scale)
            except asyncio.TimeoutError:
                # 没有接收到消息，跳过并继续等待
                pass

    async def video_stream(self, websocket, path):
        """
        循环获取视频帧并处理。
        """
        while True:
            # 获取Tello视频帧并处理
            frame1 = cv2.resize(self.tello.get_frame_read().frame, (256, 256))
            in_img = cv2.cvtColor(np.array(frame1, dtype='float32'), cv2.COLOR_RGB2BGR)
            self.input_data[0] = in_img

            # 模型预测
            job_id = self.dpu.execute_async(self.input_data, self.output_data)
            self.dpu.wait(job_id)

            # 获取模型预测后的帧
            frame2 = np.array(self.output_data).squeeze()

            # 转换颜色空间和裁剪
            rgb_frame1 = in_img
            rgb_frame2 = frame2.clip(0, 255)

            # 编码两个视频帧
            _, end1 = cv2.imencode('.jpg', rgb_frame1)
            _, end2 = cv2.imencode('.jpg', rgb_frame2)

            # 进行base64编码
            jpg1_as_text = base64.b64encode(end1).decode('utf-8')
            jpg2_as_text = base64.b64encode(end2).decode('utf-8')

            # 将两个视频帧发送给客户端
            await websocket.send(jpg1_as_text + '||' + jpg2_as_text)

    def run_video_server(self):
        """
        启动视频流 WebSocket 服务器的独立线程。
        """
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        server = websockets.serve(self.video_stream, "localhost", self.video_port)
        loop.run_until_complete(server)
        print(f"视频流服务器已启动，端口：{self.video_port}")
        loop.run_forever()

    def run_command_server(self):
        """
        启动控制指令 WebSocket 服务器的独立线程。
        """
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        server = websockets.serve(self.handle_commands, "localhost", self.command_port)
        loop.run_until_complete(server)
        print(f"控制指令服务器已启动，端口：{self.command_port}")
        loop.run_forever()

    def start(self):
        """
        使用多线程分别启动视频流和控制指令服务器。
        """
        video_thread = Thread(target=self.run_video_server)
        command_thread = Thread(target=self.run_command_server)
        video_thread.start()
        command_thread.start()
        video_thread.join()
        command_thread.join()


# 仅在Jupyter环境中运行
server = TelloDpuServer(dpu_bitfile="dpu.bit", model_file="sesr_model_best.xmodel")
server.start()

[INFO] tello.py - 131 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 439 - Send command: 'command'
[ERROR] tello.py - 459 - 'utf-8' codec can't decode byte 0xcc in position 0: invalid continuation byte
[INFO] tello.py - 439 - Send command: 'command'
[INFO] tello.py - 463 - Response command: 'ok'
[INFO] tello.py - 439 - Send command: 'streamon'
[INFO] tello.py - 463 - Response streamon: 'ok'


控制指令服务器已启动，端口：5679
视频流服务器已启动，端口：5678


[INFO] tello.py - 439 - Send command: 'takeoff'
[INFO] tello.py - 463 - Response takeoff: 'ok'
[INFO] tello.py - 439 - Send command: 'back 25'
[INFO] tello.py - 463 - Response back 25: 'ok'
[INFO] tello.py - 439 - Send command: 'forward 25'
[INFO] tello.py - 463 - Response forward 25: 'ok'
[INFO] tello.py - 439 - Send command: 'forward 25'
[INFO] tello.py - 463 - Response forward 25: 'ok'
[INFO] tello.py - 439 - Send command: 'up 25'
[INFO] tello.py - 463 - Response up 25: 'ok'
[INFO] tello.py - 439 - Send command: 'up 25'
[INFO] tello.py - 463 - Response up 25: 'ok'
[INFO] tello.py - 439 - Send command: 'down 25'
[INFO] tello.py - 463 - Response down 25: 'ok'
[INFO] tello.py - 439 - Send command: 'down 25'
[INFO] tello.py - 463 - Response down 25: 'ok'
[INFO] tello.py - 439 - Send command: 'down 25'
[INFO] tello.py - 463 - Response down 25: 'ok'
[INFO] tello.py - 439 - Send command: 'land'
[INFO] tello.py - 463 - Response land: 'ok'
